In [1]:
# !javac de/unima/ki/anyburl/*.java -d build

In [2]:
# !jar cfv AnyBURL-23-1.jar -C build .

In [3]:
import pandas as pd
import zipfile
import os
from tqdm import tqdm

In [4]:
os.cpu_count()

32

In [5]:
import psutil
psutil.virtual_memory()

svmem(total=99980713984, available=96078352384, percent=3.9, used=3438936064, free=96298008576, active=744464384, inactive=87363584, buffers=12128256, cached=231641088, shared=2748416, slab=264384512)

In [6]:
# def train(trace_id):
#     train_prop = f'PATH_TRAINING = MSCallGraph_traces/train/{trace_id}_transductive_train.tsv\nPATH_OUTPUT   = rules\nSNAPSHOTS_AT = 10\nWORKER_THREADS = 30\nSAFE_PREFIX_MODE = true'
#     f = open("train_prop.txt", "w")
#     f.write(train_prop)
#     f.close()
#     _=!java -Xmx72G -cp AnyBURL-23-1.jar de.unima.ki.anyburl.Learn train_prop.txt

In [7]:
def predict(trace_id):
    predict_prop = f'PATH_TRAINING = MSCallGraph_train_transductive.tsv\nPATH_VALID    = MSCallGraph_validation_transductive.tsv\nPATH_TEST     = MSCallGraph_traces/test/{trace_id}_transductive_test.tsv\nPATH_RULES    = rules-100\nPATH_OUTPUT   = preds-100\nWORKER_THREADS = 30\nTOP_K_OUTPUT = 100\nSAFE_PREFIX_MODE = true'
    f = open("predict_prop.txt", "w")
    f.write(predict_prop)
    f.close()
    _=!java -Xmx72G -cp AnyBURL-23-1.jar de.unima.ki.anyburl.Apply predict_prop.txt

In [8]:
def test(trace_id):
    eval_prop = f'PATH_TRAINING = MSCallGraph_train_transductive.tsv\nPATH_VALID    = MSCallGraph_validation_transductive.tsv\nPATH_TEST     = MSCallGraph_traces/test/{trace_id}_transductive_test.tsv\nPATH_PREDICTIONS = preds-100\nTOP_K = 100\nSAFE_PREFIX_MODE = true'
    f = open("eval_prop.txt", "w")
    f.write(eval_prop)
    f.close()
    to_return = !java -Xmx72G -cp AnyBURL-23-1.jar de.unima.ki.anyburl.Eval eval_prop.txt
    return to_return[-1]

In [9]:
def test_metrics(traceid):
#     train(traceid)
    predict(traceid)
    return(test(traceid))

In [10]:
def test_on_traces(model_name):
    directory = model_name+"_testing_traces"

    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f'Directory {directory} created successfully!')
    else:
        print(f'Directory {directory} already exists.')
        


    # Specifica il percorso della cartella da cui si vogliono ottenere i nomi dei file
    folder_path =  "MSCallGraph_traces/train/"

    files_list = []
    for filename in os.listdir(folder_path):
        # Ottenere il nome del file
        file_name = os.path.basename(filename)
        files_list.append(file_name)
        
    metric_names = ['hits_at_1','hits_at_3','hits_at_10','mrr']
    all_traces_list = []
    # Scansione di ogni file nella cartella
    with tqdm(desc=f'{model_name} testing traces', total=len(files_list)) as progress_bar:
        for file_name in files_list:
            # Stampa il nome del file
            all_traces_list.append(test_metrics(file_name[:-23]).split())
            progress_bar.update(1)


    trace_df = pd.DataFrame(all_traces_list,columns=metric_names)
    trace_df.to_csv(f"{model_name}_testing_traces/trace_test.csv")

    
    display(trace_df)
    def zip_folder(folder_path, output_path):
        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    zipf.write(os.path.join(root, file))

    folder_path = model_name+"_testing_traces"
    output_path = f'{folder_path}.zip'

    zip_folder(folder_path, output_path)

In [11]:
test_on_traces("anyburl_pretrained")

Directory anyburl_pretrained_testing_traces already exists.


anyburl_pretrained testing traces: 100%|██████████████████████████████████████████| 1027/1027 [1:08:45<00:00,  4.02s/it]


,hits_at_1,hits_at_3,hits_at_10,mrr
0,0.3333,0.3333,0.8333,0.4189
1,1.0000,1.0000,1.0000,1.0000
2,0.5000,0.5000,1.0000,0.5714
3,0.0000,0.0000,0.5000,0.1010
4,0.6000,0.8000,0.9000,0.7250
...,...,...,...,...
1022,0.5000,0.5000,1.0000,0.5556
1023,0.0000,1.0000,1.0000,0.5000
1024,0.3571,0.4286,0.8571,0.4859
1025,0.0000,0.5000,1.0000,0.2667
